In [8]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Import the data

In [9]:
df = pd.read_excel(r'Coffeecrunchchain.xlsx') #import the dataset
df.head(5) #get an impression of the data

,Profit,Margin,Sales,COGS,Total Expenses,Marketing,Inventory,Budget Profit,Budget Margin,Budget Sales,...,Date,Market,Lon,Lat,State,Area Code,Market Size,Product Type,Product,Type
0,140,228,456,228,88,63,1459,150,220,430,...,2012-01-01,West,-119.417931,36.778259,California,661,Major Market,Espresso,Caffe Latte,Regular
1,17,145,250,105,128,95,725,20,140,240,...,2012-01-01,West,-119.417931,36.778259,California,818,Major Market,Espresso,Caffe Mocha,Regular
2,203,312,546,234,109,77,1310,200,300,520,...,2012-01-01,West,-119.417931,36.778259,California,213,Major Market,Espresso,Decaf Espresso,Decaf
3,102,139,234,95,37,26,821,100,120,200,...,2012-01-01,West,-119.417931,36.778259,California,510,Major Market,Herbal Tea,Chamomile,Decaf
4,160,245,452,207,85,64,965,160,220,390,...,2012-01-01,West,-119.417931,36.778259,California,310,Major Market,Herbal Tea,Lemon,Decaf


# Barchart with Buttons

Create a bar chart that visualizes the monthly sales for each market (Central, East, South, and West). This bar chart has the following interactive features:

- it allows the user to select the market that is displayed,
- when hovering over the bar parts, the state and product are displayed so that the user can easily identify where the sales are coming from.

In [22]:
buttons = [] # create an empty list that will later contains the interactive buttons (in this case a drop-down menu where you can select the market)
i = 0

# create figure
fig = go.Figure()

market_list = list(df['Market'].unique()) # creates a list that includes all unique values of the Market variable



# creating Traces with Bar Charts for each Market, that can be later selected via the buttons menu
for market in market_list:
    fig.add_trace(
        go.Bar(
            x = df['Date'][df['Market']==market], # Dates and Sales are filterd regarding the markets
            y = df['Sales'][df['Market']==market], 
            name = market , visible = (i==0),
            hovertext=df['State'] + str('/') +  df['Product'] # this code part defines the text that is displayed when hovering over the bar parts
        )
    )

    
    
    
    

# Create the drop-down menu that allow the user to select a market    


## the first option, displays the data for all four markets at once    
args = [True] * len(market_list)   
button = dict(label = "All Markets",
                  method = "update",
                  args=[{"visible": args}])
buttons.append(button)


## next, we create the options to diplay individual markets
for market in market_list:
    args = [False] * len(market_list)
    args[i] = True
    
    #create a button object for the country we are on
    button = dict(label = market,
                  method = "update",
                  args=[{"visible": args}])
    
    #add the button to our list of buttons
    buttons.append(button)
    
    #i is an iterable used to tell our "args" list which value to set to True
    i+=1
    

    
fig.update_layout(updatemenus=[dict(active=0,
                                    type="buttons",
                                    direction = "left",
                                    buttons=buttons,
                                    x = 0,
                                    y = 1.01,
                                    xanchor = 'left',
                                    yanchor = 'bottom'),
                              ],
                  title_text="Sales in a Market -  Bar Chart")

fig.update_layout(
    autosize=False,
    width=1000,
    height=800,)

# Waterfall Chart with Drop-Down Menu

A waterfall chart helps in understanding the cumulative effect of sequentially introduced positive or negative values. These intermediate values can either be time-based (e.g., profit/losses over time) or category-based (e.g., sales, other income, expenses).

This section creates a waterfall chart that illustrates how profits are derived from sales, costs of goods sold, and total expenses. The plot has the following features:

- drop-down menu where the state can be selected
- comparison of 2012 and 2013 in one plot
- distinguish 2012 and 2013 by color

In [23]:
buttons = []
i=0

# create figure
fig1  = go.Figure()


# list with all states
market_list = list(df['State'].unique())


# create traces with waterfall charts for each state for 2012
for market in market_list:
    fig1.add_trace(
    go.Waterfall(                 
        x = ["Sales", "COGS","Total Expenses", "Profit"], 
        y = [int(df['Sales'][(df['State']==market) & (df['Date'].dt.year==2012)].sum()), # sum sales over all products
             int(-df['COGS'][(df['State']==market) & (df['Date'].dt.year==2012)].sum()),  
             int(-df['Total Expenses'][(df['State']==market) & (df['Date'].dt.year==2012)].sum()), 
             None],
        base = 0,
        measure = [ "relative","relative","relative","total" ], 
        # the default is "relative", "total" is used to compute sums, 
        # and "absolute" can be used to reset the computed total or to declare an initial value where needed
        name = 2012,
        decreasing = {"marker":{"color":'rgb(255,153,153)'}}, # define the color of a "relative" bar with negatoive value
        increasing = {"marker":{"color":'rgb(204,255,204)'}}, # define the color of a "relative" bar with positive value
        totals = {"marker":{"color":'rgb(204,229,255)'}} # define the color of a "totals" bar
        ))


# create traces with waterfall charts for each state for 2013    
for market in market_list:
    fig1.add_trace(
    go.Waterfall(                 
        x = ["Sales", "COGS","Total Expenses", "Profit"], 
        y = [int(df['Sales'][(df['State']==market) & (df['Date'].dt.year==2013)].sum()), 
             int(-df['COGS'][(df['State']==market) & (df['Date'].dt.year==2013)].sum()),  
             int(-df['Total Expenses'][(df['State']==market) & (df['Date'].dt.year==2013)].sum()), 
             None],
        base = 0,
        measure = [ "relative","relative","relative","total" ],
        name = 2013 
        ))
   
    


    
    

## Create the options to diplay individual markets
for market in market_list:
    args = [False] * 2*len(market_list)
    args[i] = True # 2012 trace 
    args[i+20] = True # corresponding 2013 trace
    
    #create a button object for the country we are on
    button = dict(label = market,
                  method = "update",
                  args=[{"visible": args}])
    
    #add the button to our list of buttons
    buttons.append(button)
    
    #i is an iterable used to tell our "args" list which value to set to True
    i+=1
    

    
fig1.update_layout(updatemenus=[dict(active=0,
                                    type="dropdown", # option to specift that button is a drop-down menu
                                    buttons=buttons,
                                    x = 0,
                                    y = 1.02,
                                    xanchor = 'left',
                                    yanchor = 'bottom'),
                              ],
                  title_text="Profit Waterfall Chart")

fig1.update_layout(
    autosize=False,
    width=1000,
    height=800,)
                               

fig1.show()

# Pie Chart with Drop-Down Menu

This section creates a pie chart that shows how each product contributes to the states' total profits. The plot has the following features:

- a drop-down menu that allows the user to select a state,
- comparison of 2012 and 2013 with subplots next to each other,
- size of the pie represents total profits,
- when hovering over a pie piece, the year, product name, total profits from that product, and profit contribution in percent are displayed.

In [24]:
buttons = []
i=0

# create figure with subplots
fig1 = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])

# list with all states
market_list = list(df['State'].unique())


# traces with pie charts for each state for 2012
for market in market_list:
    fig1.add_trace(
        go.Pie(
            labels=df['Product'][(df['State']==market) & (df['Date'].dt.year==2012)], 
            values=df['Profit'][(df['State']==market) & (df['Date'].dt.year==2012)], name="2012", 
            hole=0.3, # make a hole in the middle of the pie
            scalegroup='one', # pie size represents total profits
        ),
              1, 1)

# traces with pie charts for each state for 2013    
for market in market_list:
        fig1.add_trace(
        go.Pie(
            labels=df['Product'][(df['State']==market) & (df['Date'].dt.year==2013)], 
            values=df['Profit'][(df['State']==market) & (df['Date'].dt.year==2013)], name="2013", hole=0.3,scalegroup='one'),
              1, 2)
   
    
    

## create the options to diplay individual states
for market in market_list:
    args = [False] * 2*len(market_list)
    args[i] = True # trace with 2012 pie
    args[i+20] = True #  trace with corresponding 2013 pie
    
    #create a button object for the state we are on
    button = dict(label = market,
                  method = "update",
                  args=[{"visible": args}])
    
    #add the button to our list of buttons
    buttons.append(button)
    
    #i is an iterable used to tell our "args" list which value to set to True
    i+=1
    

    
fig1.update_layout(updatemenus=[dict(active=0,
                                    type="dropdown",
                                    buttons=buttons,
                                    x = 0,
                                    y = 1.02,
                                    xanchor = 'left',
                                    yanchor = 'bottom'),
                              ])


fig.update_layout(
    autosize=False,
    width=1000,
    height=800)


# create Pie title and labels in the middle of the pies
fig1.update_layout(
    title_text="Profit Composition - Pie Chart",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='2012', x=0.183, y=0.5, font_size=20, showarrow=False),
                 dict(text='2013', x=0.818, y=0.5, font_size=20, showarrow=False)])    


fig1.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=1.1
))


fig1.show()

# Time-Series Plot with Range Slider and Selectors

Create a plot that visualizes the time series of sales for the Amaretto Product in Colorado. The plot has the following interactive features

- buttons to select the time period displayed,
- range slider,
- displays date and sales number when hovering over a data point.

In [25]:
# Create figure
fig2 = go.Figure()

fig2.add_trace(
    go.Scatter(x=list(df['Date'][(df['State']=='Colorado') & (df['Product']=='Amaretto')]), 
               y=list(df['Sales'][(df['State']=='Colorado') & (df['Product']=='Amaretto')])))

# Set title
fig2.update_layout(
    title_text="Time series with range slider and selectors"
)

# Add range slider and buttons
fig2.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig2.show()

# Stacked Time Series Plots with Range Slider

Create stacked time series plots that visualize the sales of 5 products (Amaretto, Columbian, Decaf Irish Cream, Green Tea, and Caffe Mocha) in Colorado. Note especially the following properties of the graph:

- the time series plots are stacked
- products are visually separated by color saturation
- years are visually separated by color (blue=2012, green=2013)
- interactive range slider
- product name and sales number are shown when hovering over data points
- the axis range is the same for all subplots so that comparisons are not biased

In [14]:
df['Sales'][(df['State']=='Colorado')].max() #determine the max value for y-axis

387

In [26]:
# Create figure
fig3 = go.Figure()


product_list = ["Amaretto","Colombian","Decaf Irish Cream","Green Tea","Caffe Mocha"] # creates a list that includes all unique values of the Market variable
i=1


# create trace for each product
for product in product_list:
    fig3.add_trace(go.Scatter(
    x=list(df['Date'][(df['State']=='Colorado') & (df['Product']==product)]),
    y=list(df['Sales'][(df['State']=='Colorado') & (df['Product']==product)]),
    name=product,
    text=list(df['Sales'][(df['State']=='Colorado') & (df['Product']==product)]),
    yaxis="y"+str(i),
    ))
    i=i+1


# style all the traces
fig3.update_traces(
    hoverinfo="name+x+text",
    line={"width": 0.5},
    marker={"size": 8},
    mode="lines+markers", # data points are marked with dots
    showlegend=False
)


# Add shapes - blue and green background for each year
fig3.update_layout(
    shapes=[
        dict(
            fillcolor="rgba(63, 81, 181, 0.3)",
            line={"width": 0},
            type="rect",
            x0="2012-01-01",
            x1="2012-12-31",
            xref="x",
            y0=0,
            y1=0.2,
            yref="paper"
        ),
        dict(
            fillcolor="rgba(63, 81, 181, 0.25)",
            line={"width": 0},
            type="rect",
            x0="2012-01-01",
            x1="2012-12-31",
            xref="x",
            y0=0.2,
            y1=0.4,
            yref="paper"
        ),
        dict(
            fillcolor="rgba(63, 81, 181, 0.2)",
            line={"width": 0},
            type="rect",
            x0="2012-01-01",
            x1="2012-12-31",
            xref="x",
            y0=0.4,
            y1=0.6,
            yref="paper"
        ),
        dict(
            fillcolor="rgba(63, 81, 181, 0.15)",
            line={"width": 0},
            type="rect",
            x0="2012-01-01",
            x1="2012-12-31",
            xref="x",
            y0=0.6,
            y1=0.8,
            yref="paper"
        ),
        dict(
            fillcolor="rgba(63, 81, 181, 0.1)",
            line={"width": 0},
            type="rect",
            x0="2012-01-01",
            x1="2012-12-31",
            xref="x",
            y0=0.8,
            y1=1,
            yref="paper"
        ),
        dict(
            fillcolor="rgba(76, 175, 80, 0.3)",
            line={"width": 0},
            type="rect",
            x0="2013-01-01",
            x1="2013-12-31",
            xref="x",
            y0=0,
            y1=0.2,
            yref="paper"
        ),
        dict(
            fillcolor="rgba(76, 175, 80, 0.25)",
            line={"width": 0},
            type="rect",
            x0="2013-01-01",
            x1="2013-12-31",
            xref="x",
            y0=0.2,
            y1=0.4,
            yref="paper"
        ),
        dict(
            fillcolor="rgba(76, 175, 80, 0.2)",
            line={"width": 0},
            type="rect",
            x0="2013-01-01",
            x1="2013-12-31",
            xref="x",
            y0=0.4,
            y1=0.6,
            yref="paper"
        ),
        dict(
            fillcolor="rgba(76, 175, 80, 0.15)",
            line={"width": 0},
            type="rect",
            x0="2013-01-01",
            x1="2013-12-31",
            xref="x",
            y0=0.6,
            y1=0.8,
            yref="paper"
        ),
        dict(
            fillcolor="rgba(76, 175, 80, 0.1)",
            line={"width": 0},
            type="rect",
            x0="2013-01-01",
            x1="2013-12-31",
            xref="x",
            y0=0.8,
            y1=1,
            yref="paper"
        )
    ]
)






# Update axes
fig3.update_layout(
    xaxis=dict(
        autorange=True,
        rangeslider=dict(
            autorange=True,
        ),
        type="date"
    ),
    yaxis=dict(
        domain=[0.04, 0.2],
        linecolor="#673ab7",
        mirror=True, #creates y axis line on the left side
        range=[0, 400],
        side="right",
        tickfont={"color": "#673ab7"},
        title="Amaretto",
        titlefont={"color": "#673ab7"}
    ),
    yaxis2=dict(
        domain=[0.24, 0.4],
        linecolor="#E91E63",
        mirror=True,
        range=[0,400],
        side="right",
        tickfont={"color": "#E91E63"},
        title="Colombian",
        titlefont={"color": "#E91E63"}
    ),
    yaxis3=dict(
        domain=[0.44, 0.6],
        linecolor="#795548",
        mirror=True,
        range=[0,400],
        side="right",
        tickfont={"color": "#795548"},
        title="Decaf",
        titlefont={"color": "#795548"}
    ),
    yaxis4=dict(
        domain=[0.64, 0.8],
        linecolor="#607d8b",
        mirror=True,
        range=[0,400],
        side="right",
        tickfont={"color": "#607d8b"},
        title="Green Tea",
        titlefont={"color": "#607d8b"}
    ),
    yaxis5=dict(
        domain=[0.84, 1],
        linecolor="#2196F3",
        mirror=True, 
        range=[0,400],
        side="right",
        tickfont={"color": "#2196F3"},
        title="Caffe Mocha",
        titlefont={"color": "#2196F3"}
    )
)

# Update layout
fig3.update_layout(
    hovermode="x", #multiple hoverlables appear (one for the date and one with the sales number and product)
    height=800,
    template="plotly_white",
    margin=dict(
        t=100,
        b=100
    ),
)


# Title
fig3.update_layout(
    title_text="Stacked Time Series Plots with Range Slider"
)

fig3.show()

# Bubble Map

A Bubble Map uses circles of different sizes and colors to represent a numeric value in a territory. This section creates a bubble map that illustrates the profits per state in the year 2012. The map has the following features:

- when hovering over a bubble the state name and profits are displayed
- Zoom-function
- amount of profits are represented by bubble size and color
- by clicking on the legend, bubble categories can be made invisble or visible

Prepare the dataset for the bubble map. Create a dataset that contains the sum of profits for each state in 2012 and the coordinates (variables: Lon, Lat).

In [16]:
df['Year']=df['Date'].dt.year
df2=df[['Profit', 'State', 'Year', 'Lon','Lat']]
df2=df2[df2['Year']==2012]
df3 = pd.DataFrame(df2.groupby(by=['State','Year', 'Lon','Lat']).sum()).reset_index()
df3.head(5)

,State,Year,Lon,Lat,Profit
0,California,2012,-119.417931,36.778259,12964
1,Colorado,2012,-105.358887,39.113014,7227
2,Connecticut,2012,-72.699997,41.599998,3093
3,Florida,2012,-81.760254,27.994402,5029
4,Illinois,2012,-89.000000,40.000000,12577


The summary descriptives of the Profit variable are helpful when deciding about the color categories for the bubble map.

In [27]:
df3['Profit'].describe() 

count       20.000000
mean      5290.500000
std       3414.610211
min        330.000000
25%       3139.500000
50%       4512.500000
75%       6840.750000
max      12964.000000
Name: Profit, dtype: float64

In [28]:
df3['text'] = df3['State'] + str('/Profit:') +  df3['Profit'].map(str) #  text that is displayed when hovering over a bubble
limits = [(0,500),(501,3000),(3001,4500),(4501,7000),(7000,15000)] # profit thresholds for color categories
colors = ["red","orange","lightgrey","lightseagreen","green"] # corresponding colors to the thresholds


# create figure
fig = go.Figure()

# create traces for each color categorie
for i in range(len(limits)):
    lim = limits[i]
    df_sub = df3[(df3['Profit']>=lim[0]) &(df3['Profit']<=lim[1]) ]
    fig.add_trace(go.Scattergeo(
        locationmode = 'USA-states',
        lon = df_sub['Lon'], # longtitude
        lat = df_sub['Lat'], # latitude
        text = df_sub['text'], # text that is displayed when hovering
        marker = dict(
            size = df_sub['Profit']/5, # bubble size represent scaled profits (scaled by 5 since the bubble are otherwise too large)
            color = colors[i],
            line_color='rgb(40,40,40)',
            line_width=0.5,
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1]))) 

# figure layout
fig.update_layout(
        title_text = 'Profits per State in 2012',
        showlegend = True,
        geo = dict(
            scope = 'usa',
            landcolor = 'rgb(217, 217, 217)',
        )
    )

fig.show()

# Interactive Pivot Tables and Plots with PivotTable.js

PivotTable.js is a Javascript Pivot Table library with drag'n'drop functionality for creating Pivottables and other plots.

If you have not installed the PivotTable.js package yet, do so by running the following command

- !pip install pivottablejs

In [31]:
from pivottablejs import pivot_ui

pivot_ui(df)

# create a Pivot table where the sum of profits is displayed per market and product type